In [2]:
import numpy as np
import math as m
from scipy.stats import norm

###  Black Scholes Formula

In [5]:
def black_scholes(S0, E, r,t,vol, option):
    
    d1 = ( np.log(S0/E) + (r+(vol**2/2))*t )/(vol*np.sqrt(t))
    d2 = ( np.log(S0/E) + (r-(vol**2/2))*t )/(vol*np.sqrt(t))
    
    if option == 'call' or option == 'Call':
        payoff = S0* norm.cdf(d1) - E*np.exp(-r*t)*norm.cdf(d2)
    else:
        payoff =  E*np.exp(-r*t)*(1- norm.cdf(d2)) - S0*(1-norm.cdf(d1))
    
    return payoff
        
    
#put call -- C + e^-rt*E  = P + S

In [10]:
T = [.25,.5,.75,1]
for t in T:
    print(black_scholes(5,10,.12,t,.5,'put'))
    

4.70659485121881
4.452669135807984
4.246542116278227
4.073262248678926


### Binomial Tree for American options

In [27]:
def intrinsic_put(S0,E,M,i, dpwrs,upwrs):
    siValues = S0*upwrs[range(0,i+1)] *dpwrs[range(M-i,M+1)]
    return np.maximum(E-siValues,0)
    
def dFac(r, dt): #discount factor
    return np.exp(-r * dt)

In [69]:
def American_BinomialTree(E, S0, M, T, r, vol, option = 'call', risk_neutrality = 0):
    """
    E is our strike price
    S0 is initial price
    M is the number of steps
    T is our time to expiry
    r is our risk free rate
    vol is the volatility of the asset
    Choose option to be 'call' or 'put' to decide on the type of option wanted
    IF risk_neutrality = 0 we dont preserve it, if == 1 we do & if ==2 we use the Cox Ross Rubenstein Formula
    
    """

    dt = T/M
    

    #don't preserve risk neutrality
    if risk_neutrality == 0:
        u = np.exp(vol*np.sqrt(dt) + (r-vol**2/2)*dt)
        d = np.exp(-vol*np.sqrt(dt) + (r-vol**2/2)*dt)
        p =1/2
        
        
        
    #Preserve risk neutrality
    elif risk_neutrality == 1:
        A = 1/2 * (np.exp((r+vol**2)*dt) +  np.exp(-r*dt))
        u = A + np.sqrt(A**2 -1)
        d = A - np.sqrt(A**2 -1)
        p = (np.exp(r*dt)-d)/(u-d)
    
    #Cox Ross Rubenstein formula
    elif risk_neutrality == 2:
        u = np.exp(vol* np.sqrt(dt))
        d = np.exp(-vol* np.sqrt(dt))
        p = (np.exp(r*dt)-d )/(u-d)
        
    upwrs = u**np.arange(0,M+1)
    dpwrs = d**np.arange(M,-1,-1)
            

    
    assetAtExpiry = (S0*d**np.arange(M,-1,-1)) * (u**np.arange(0,M+1,1))
    
    putVals = np.maximum(E - assetAtExpiry, 0)
    callVals = np.maximum(assetAtExpiry - E, 0)
    
    if option == 'call' or option == 'Call':
        optVal = np.maximum(assetAtExpiry-E,0)
    else:
        optVal = np.maximum(E- assetAtExpiry, 0)
        

        
    #This is the stage that needs to change for the American option
    
    #Now compute the value of the call option at each stage
    for i in range(int(M)-1,-1,-1):
        valsUp = putVals[range(1,i+2)]
        valsDown = putVals[range(0,i+1)]
        putVals = np.maximum(intrinsic_put(S0,E,M,i,dpwrs,upwrs), dFac(r,dt) * (p*valsUp + (1-p)*valsDown))
    return putVals

In [70]:
T = [.25,.5,.75,1]
for t in T:
    print(American_BinomialTree(E= 10, S0 = 9,M =  256 ,T =  t,r = .06, vol =.3, option = 'put', risk_neutrality = 2),
         American_BinomialTree(E= 10, S0 = 9,M =  256 ,T =  t,r = .06, vol =.3, option = 'put', risk_neutrality = 1),
         American_BinomialTree(E= 10, S0 = 9,M =  256 ,T =  t,r = .06, vol =.3, option = 'put', risk_neutrality = 0))

[1.12596764] [1.12599642] [1.12601313]
[1.25454005] [1.25463288] [1.25521853]
[1.35450496] [1.35467736] [1.35399375]
[1.43466237] [1.43493473] [1.43522462]


In [67]:
#first set up values corresponding to powers of d and u
S0 = 10
u = 1.1
d= .9
M = 4
upowers = u**np.arange(0,M+1)
dpowers = d**np.arange(M,-1,-1)

i =3
S0*upowers[range(0,i+1)] * dpowers[range(M-i,M+1)]


array([ 7.29,  8.91, 10.89, 13.31])